In [1]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [4]:
@memoized
def coef(n):
    return Fraction(1, 6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [6]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [7]:
def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == (y,x,z):
            return read(H_YXZ_CACHE, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, max_n=n)[n]

# General constraints

In [181]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat

In [182]:
n = 3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
num_pos_per_n

[0, 5, 31]

In [183]:
num_pos = sum(num_pos_per_n)  # number of positive compound indicators
num_neg = sum(num_neg_per_n)
num_pos

36

In [184]:
pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])

In [185]:
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
const

Fraction(1, 6)

In [186]:
all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_pos_comp_inds

[[3*x <= y],
 [2*x <= z],
 [2*x <= y],
 [3*x <= z],
 [x <= -y + z],
 [7*x <= y],
 [4*x <= z, 3*x < y],
 [5*x <= y],
 [6*x <= z],
 [5*x <= 3*y, y < 3*x],
 [3*x <= y + z, y < 3*x],
 [2*x <= y],
 [x <= -y + z],
 [3*x <= y + z, z < 2*x],
 [z < 2*x],
 [6*x <= y],
 [5*x <= z, 2*x < y],
 [4*x <= y],
 [7*x <= z],
 [y < 2*x],
 [3*x <= y + z, y < 2*x],
 [x < -y + z],
 [3*x <= -y + z],
 [3*x <= y + z, z < 3*x],
 [5*x <= 3*z, z < 3*x],
 [2*x <= z, -y + z < x],
 [3*x <= 2*y],
 [2*x <= -y + z],
 [x < -y + z],
 [3*x <= -y + z],
 [3*y < z],
 [x <= -3*y + z],
 [-y + z < x],
 [x <= -3*y + 3*z, -y + z < x],
 [x <= 3*y - z],
 [x <= -2*y + 2*z, z < 3*y]]

In [187]:
model = gp.Model(env=gp.Env())
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
model.Params.MIPFocus = 1
# model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-9
# model.Params.IntFeasTol = 1e-9

gx = model.addVar(lb=1, vtype='I', name='x')
gy = model.addVar(lb=2, vtype='I', name='y')
gz = model.addVar(lb=3, vtype='I', name='z')

# gx = model.addVar(lb=1, name='x')
# gy = model.addVar(lb=2, name='y')
# gz = model.addVar(lb=3, name='z')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1


In [188]:
from sympy import symbols, Add
from gurobipy import Model, LinExpr, and_

# Define SymPy variables
x,y,z = symbols('x y z')

# Mapping between SymPy and Gurobi variables
SP_TO_GP = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=SP_TO_GP):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(to_gp_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [189]:
EPS = 1
def unify_ineq(ineq):
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(lhs - rhs, 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= EPS
        return Rel(lhs - rhs - EPS, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(rhs - lhs, 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= EPS
        return Rel(rhs - lhs - EPS, 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [191]:
unify_ineq(-3*x>1)

-3*x - 2 >= 0

In [192]:
def add_ind_constr(ineq, name='ind', var_map=SP_TO_GP):
    ''' Add indicator constraint for a single inequality'''
    r = unify_ineq(ineq)  # to the form >= 0
    lhs = to_gp_linexpr(r.lhs, var_map=var_map)
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -EPS))  # lhs < 0
    return v

In [193]:
# for each compound indicator group, restrict their sum to be [0,1]
def add_ind_constrs(*ineqs, var_map=SP_TO_GP):
    ''' Adds variables and constraints for a compound indicator. '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], name=f'single_ind_{str(ineqs[0])}', var_map=var_map),)
    
    I = model.addVar(vtype=GRB.BINARY, name=f'main_ind_{str(ineqs)}')
    component_vars = [add_ind_constr(i, name=f'component_ind_{str(i)}', var_map=var_map) for i in ineqs]
    model.addConstr(I == and_(component_vars), name=f'andconstr_{str(ineqs)}')
    return I, component_vars

In [194]:
pos_inds = [add_ind_constrs(*i) for i in all_pos_comp_inds]
neg_inds = [add_ind_constrs(*i) for i in all_neg_comp_inds]
pos_inds

[(<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),


In [195]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)

In [196]:
model.setObjective(obj, GRB.MINIMIZE)
# f_constr = model.addConstr(obj <= 0, name='non_pos_f')
# constraints for V
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [197]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 111 columns and 4 nonzeros
Model fingerprint: 0x4040817d
Model has 195 general constraints
Variable types: 0 continuous, 111 integer (108 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 7e+00]
Presolve added 292 rows and 261 columns
Presolve time: 0.01s
Presolved: 294 rows, 372 columns, 841 nonzeros
Presolved model has 174 SOS constraint(s)
Variable types: 0 continuous, 372 integer (195 binary)
Found heuristic solution: objective 0.1037037

Root relaxation: objective -1.648148e-01, 158 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |   

In [198]:
def analyse_result(m=model):
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found:")
        for v in model.getVars():
            print(f"{v.varName}: {v.x}, {not (not v.x)}")
        print(f"Objective value: {model.objVal}")
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the bounds and try to make the model feasible
        print('The model is infeasible; relaxing the bounds')
        # can only relax thresh constraint
        relaxable_vars = [gx, gy, gz]
        pen = [1] * len(relaxable_vars)
        model.feasRelax(1, False, 
                        relaxable_vars, pen, pen, 
                        [f_constr], 
                        [1])
        model.NODELIMIT = 100000
        model.optimize()

In [199]:
analyse_result()

Optimal solution found:
x: 4.0, True
y: 5.0, True
z: 6.0, True
single_ind_3*x <= y: -0.0, False
single_ind_2*x <= z: -0.0, False
single_ind_2*x <= y: -0.0, False
single_ind_3*x <= z: -0.0, False
single_ind_x <= -y + z: -0.0, False
single_ind_7*x <= y: -0.0, False
main_ind_(4*x <= z, 3*x < y): -0.0, False
component_ind_4*x <= z: -0.0, False
component_ind_3*x < y: -0.0, False
single_ind_5*x <= y: -0.0, False
single_ind_6*x <= z: -0.0, False
main_ind_(5*x <= 3*y, y < 3*x): -0.0, False
component_ind_5*x <= 3*y: -0.0, False
component_ind_y < 3*x: 1.0, True
main_ind_(3*x <= y + z, y < 3*x): -0.0, False
component_ind_3*x <= y + z: -0.0, False
component_ind_y < 3*x: 1.0, True
single_ind_2*x <= y: -0.0, False
single_ind_x <= -y + z: -0.0, False
main_ind_(3*x <= y + z, z < 2*x): -0.0, False
component_ind_3*x <= y + z: -0.0, False
component_ind_z < 2*x: 1.0, True
single_ind_z < 2*x: 1.0, True
single_ind_6*x <= y: -0.0, False
main_ind_(5*x <= z, 2*x < y): -0.0, False
component_ind_5*x <= z: -0.0, 

In [200]:
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [202]:
eval_f(3,4,5,6)

-0.016666666666666677

In [203]:
model.write('modelB.lp')

# LB on h_n(x,y,z) - h_n(y,x,z)

In [204]:
n = 4
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
model = gp.Model(env=gp.Env())
model.reset(1)
gx = model.addVar(lb=1, vtype='I', name='x')
gy = model.addVar(lb=2, vtype='I', name='y')
gz = model.addVar(lb=3, vtype='I', name='z')
SP_TO_GP = {x: gx, y: gy, z: gz}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Discarded solution information including additional information


In [205]:
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [206]:
pos_inds = [add_ind_constrs(*i, var_map=SP_TO_GP) for i in H_XYZ[n][1]]
neg_inds = [add_ind_constrs(*i, var_map=SP_TO_GP) for i in H_YXZ[n][1]]
const = H_XYZ[n][0] - H_YXZ[n][0]

In [207]:
obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 799 columns and 4 nonzeros
Model fingerprint: 0x0e0189e7
Model has 1370 general constraints
Variable types: 0 continuous, 799 integer (796 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2073 rows and 1722 columns
Presolve time: 0.03s
Presolved: 2075 rows, 2521 columns, 6037 nonzeros
Presolved model has 1148 SOS constraint(s)
Variable types: 0 continuous, 2521 integer (1370 binary)
Found heuristic solution: objective -0.0007716
Found heuristic solution: objective -0.0015432

Root relaxation: objective -1.118827e-01, 1204 iterations, 0.01 seconds (0.00 work units)

    Node

In [208]:
analyse_result()

Optimal solution found:
x: 7.0, True
y: 8.0, True
z: 23.0, True
single_ind_15*x <= y: -0.0, False
main_ind_(8*x <= z, 7*x < y): -0.0, False
component_ind_8*x <= z: -0.0, False
component_ind_7*x < y: 0.0, False
single_ind_11*x <= y: -0.0, False
main_ind_(12*x <= z, 3*x < y): -0.0, False
component_ind_12*x <= z: 0.0, False
component_ind_3*x < y: -0.0, False
main_ind_(13*x <= 3*y, y < 7*x): -0.0, False
component_ind_13*x <= 3*y: -0.0, False
component_ind_y < 7*x: 1.0, True
main_ind_(7*x <= y + z, y < 7*x, 3*x < y): -0.0, False
component_ind_7*x <= y + z: -0.0, False
component_ind_y < 7*x: 1.0, True
component_ind_3*x < y: -0.0, False
single_ind_5*x <= y: -0.0, False
main_ind_(x <= -y + z, 3*x < y): -0.0, False
component_ind_x <= -y + z: 1.0, True
component_ind_3*x < y: -0.0, False
main_ind_(7*x <= y + z, z < 4*x, 3*x < y): -0.0, False
component_ind_7*x <= y + z: 0.0, False
component_ind_z < 4*x: 1.0, True
component_ind_3*x < y: -0.0, False
main_ind_(4*x <= 3*z, z < 4*x, 3*x < y): -0.0, Fal

In [209]:
def eval_dh(n,a,b,c):
    vals = {x:a, y:b, z:c}
    pos_c, pos_rs = H_XYZ[n]
    neg_c, neg_rs = H_YXZ[n]
    return coef(n) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))

In [211]:
float(eval_dh(4,7,8,23))

-0.016975308641975308

In [212]:
eval_f(3,4,5,6) + eval_dh(4,7,8,23) + thresh(4)

0.016358024691358018